In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

def accuracy(outputs, targets):
    with torch.no_grad():
        predicted = outputs
        correct = predicted.eq(targets).sum().item()
        accuracy = correct / targets.numel()
    return accuracy

def precision(outputs, targets):
    with torch.no_grad():
        predicted = outputs
        true_positives = (predicted * targets).sum().item()
        false_positives = ((predicted == 1) & (targets == 0)).sum().item()
        precision = true_positives / (true_positives + false_positives)
    return precision

loss_list=[]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_classifier.to(device)
print(device)
# Определяем количество эпох
num_epochs = 30
#model.train()
criterion = nn.BCELoss()
optimizer = optim.Adam(model_classifier.parameters(), lr=0.00005)

best_loss = float('inf')
for epoch in range(num_epochs):
    running_loss = 0.0
    run_acc=0.0
    prec_acc=0.0
    model_classifier.train()
    for data in tqdm(train_dataloader):
        
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()

        # Прогоняем входные данные через автоэнкодер
        outputs = model_classifier(inputs)

        # Вычисляем функцию потерь
        loss = criterion(outputs, labels.unsqueeze(1))
        #prec = precision(outputs, labels.unsqueeze(1))
        #acc = accuracy(outputs, labels.unsqueeze(1))
        # Обратный проход и оптимизация параметров
        loss.backward()
        optimizer.step()

        # Суммируем потери по батчам
        #run_acc+=acc* inputs.size(0)
        #prec_acc+=prec* inputs.size(0)
        running_loss += loss.item() * inputs.size(0)
    # test loop
    val_loss = 0
    model_classifier.eval()
    with torch.no_grad():
        test_loss = 0.0
        run_acc_test=0.0
        prec_acc_test=0.0
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model_classifier(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            #prec = precision(outputs, labels.unsqueeze(1))
            #acc = accuracy(outputs, labels.unsqueeze(1))
            test_loss += loss.item()
            #run_acc_test+=acc* inputs.size(0)
            #prec_acc_test+=prec* inputs.size(0)
        test_loss /= len(test_loader)
        #run_acc_test/=len(test_loader)
        #prec_acc_test/=len(test_loader)
        
        print('Test Loss: {:.6f}'.format(test_loss))
    # Выводим средние потери за эпоху
    epoch_loss = running_loss / len(dataset)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch +
          1, num_epochs, epoch_loss))
    if epoch>25:
        if test_loss < best_loss:
            best_loss=test_loss
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model_classifier.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }
            torch.save(checkpoint, 'checkpoint_class1_'+str(epoch).zfill(3)+'.pth')
        
    loss_list.append([epoch,running_loss,test_loss])